In [1]:
import os
import pandas as pd
import math
import random
import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def makedir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
savedir = 'data_processed'
makedir(savedir)

## Get date of collection

### Date of collection 2023

In [7]:
doc_df = pd.read_excel('train_2023/2023/DataPublication_final/GroundTruth/DateofCollection.xlsx')
doc_2023 = {}
for index, row in doc_df.iterrows():
    if row['Location'] not in doc_2023.keys():
        doc_2023[row['Location']] = {}
    doc_2023[row['Location']][row['time']] = row['Date']
save_path = os.path.join(savedir, 'doc_2023.pkl')
f = open(save_path,'wb')
pickle.dump(doc_2023,f)
f.close()

## Get GT data

In [9]:
data_list = []
columns = ['year', 'location', 'irrigationProvided', 'experiment', 'range', 'row', 'N_level', 'genotype', 'planting_date']+['location_Ames', 'location_Crawfordsville', 'location_Lincoln', 'location_MOValley'] + ['N_level_High', 'N_level_Low', 'N_level_Medium']

### 2023 Val data

In [11]:
df_test_2023 = pd.read_csv('test_HIPS_HYBRIDS_2023_V2.3.csv')
print(df_test_2023['plotLength'].value_counts())

plotLength
17.5    288
Name: count, dtype: int64


In [12]:
df_test_2023 = df_test_2023.drop(columns = ['qrCode','poundsOfNitrogenPerAcre','plotLength', 'block','plotNumber','totalStandCount', 'daysToAnthesis', 'GDDToAnthesis', 'yieldPerAcre'])

In [13]:
df_test_2023.head()

,location,irrigationProvided,nitrogenTreatment,experiment,row,range,genotype,plantingDate
0,Ames,0,Low,2315,15,9,LH185 X LH145,5/19/23
1,Ames,0,Low,2315,11,7,PHN46 X PHK56,5/19/23
2,Ames,0,Low,2315,17,4,PHN46 X W606S,5/19/23
3,Ames,0,Low,2315,17,20,B73 X PHZ51,5/19/23
4,Ames,0,Low,2315,13,19,LH145 X LH162,5/19/23


In [14]:
for index, row in df_test_2023.iterrows():
    new_row = []
    new_row.append('2023')
    new_row.append(row['location'])
    new_row.append(row['irrigationProvided'])
    new_row.append(row['experiment'])
    new_row.append(row['range'])
    new_row.append(row['row'])
    new_row.append(row['nitrogenTreatment'])
    new_row.append(row['genotype'])
    new_row.append(row['plantingDate'])
    new_row += [1.0, 0.0, 0.0, 0.0]
    if row['nitrogenTreatment'] == 'Low':
        new_row += [0.0, 1.0, 0.0]
    if row['nitrogenTreatment'] == 'High':
        new_row += [1.0, 0.0, 0.0]
    if row['nitrogenTreatment'] == 'Medium':
        new_row += [0.0, 0.0, 1.0]
    data_list.append(new_row)

In [15]:
total_count = len(data_list)
data_split_dict = {'columns': columns, 'val': data_list}

f = open(os.path.join(savedir,'test_2023.pkl'), 'wb')
pickle.dump(data_split_dict,f)
f.close()

In [16]:
f = open(os.path.join(savedir,'test_2023.csv'),'w')
f.write(','.join(columns)+'\n')
for line in data_split_dict['val']:
    f.write(','.join([str(l) for l in line])+'\n')
f.close()